Here we'll show examples of:
- arranging plots on a grid
- Making custom plots
- And some other things

In [368]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from utility_programs.read_routines import GITM, SAMI

from utility_programs import filters, plot_help

import argparse
from tqdm.auto import tqdm
from utility_programs.plot_help import UT_from_Storm_onset
import utility_programs.plotting_routines as pr
from utility_programs import filters
import os
import datetime
import pandas as pd

from scipy import signal

import aacgmv2

import importlib

import geopandas
world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))


In [385]:
pr = importlib.reload(utility_programs.plotting_routines)
filters = importlib.reload(utility_programs.filters)

In [18]:
dtime_storm_start=datetime.datetime(2011,5,21,12,0)
# we'll need this a lot.

In [2]:
# test getting figs back from make_map...

In [3]:
gitm_dtimes, gitmgrid, gitmbins, gitmvars = GITM.read_gitm_into_nparrays(
    "/home/axb170054/scratch/GITM-testing/test_folders/step_function_driving/run/data", dtime_storm_start, t_start_idx=1, t_end_idx=1,return_vars=True)

  0%|          | 0/24 [00:00<?, ?it/s]

In [4]:
gitmbins.shape

(24, 37, 90, 180, 50)

In [9]:
nrows = 2
ncols = 2

i=0

for row in range(nrows):
    for col in range(ncols):
        
        i
        print(i, row, col)
        i+=1

0 0 0
1 0 1
2 1 0
3 1 1


In [69]:
nrows = 2
ncols = 2


data=[[0,0],[0,0]]
data[0][0] = gitmbins[0,0,:,:,30]
data[0][1] = gitmbins[5,0,:,:,30]
data[1][0] = gitmbins[10,0,:,:,30]
data[1][1] = gitmbins[15,0,:,:,30]

titles=[[0,0],[0,0]]
titles[0][0] = "1"
titles[0][1] = "2"
titles[1][0] = "3"
titles[1][1] = "4"


# Do as shown above or (depending on your needs):
"""
data = []
titles = []

numtimes = [0,5,10,15]
titles = ['one','two','three','four']
var_idx = 0

i=0
for row in range(nrows):
    for col in range(ncols):
        
        
        data[row][col] = gitmbins[numtimes[i],var_idx,:,:,30]
        titles[row][col] = titles[i]
        i+=1
"""


fig, axes = plt.subplots(nrows,ncols, figsize=(10,5), sharex=True)

for row in range(nrows):
    for col in range(ncols):
        im = pr.draw_map(data[row][col], [np.min(data1), np.max(data1)], save_or_show="return", ax=axes[row,col], title=str(row))
        
        axes[row,col].set_title(titles[row][col])
        
        if col == 0:
            axes[row,col].set_ylabel("Latitude (deg)")
        if row == nrows-1:
            axes[row,col].set_xlabel("Longitude (deg)")


fig.suptitle('Some Maps          ', fontsize=16, ha='center')
# spaces needed to center properly with the colorbar existing.
    
fig.colorbar(im, ax=axes.ravel().tolist(), orientation="vertical",
             label = 'Rho (n/m^3)')
fig.align_xlabels()


plt.show()
plt.close()

IndexError: index 30 is out of bounds for axis 4 with size 1

## Panel of Joule Heating & Particle Heating 10 mins after storm onset.

This needs to be done a little differently since the data isn't split into hemisphereres naturally

In [3]:
gitm_dtimes, gitmgrid, gitmbins, gitmvars = GITM.read_gitm_into_nparrays(
    "/glade/u/home/abukowski/scratch/GITM-simstorm-run1/run2/UA/data", , t_start_idx=1, t_end_idx=1,return_vars=True, gitm_file_pattern='2DANC*.bin')

100%|██████████| 24/24 [00:00<00:00, 24.72it/s]


In [13]:
gitmbins.shape

(24, 12, 90, 180, 1)

In [38]:
lats = np.unique(gitmgrid['latitude'])
lons = np.unique(gitmgrid['longitude'])
new_shape = [len(gitm_dtimes), len(lons), len(lats)]

maskNorth = ((lats > 45))
maskSouth = ((lats < -45))

In [39]:
gitmvars

['LocalTime',
 'SolarZenithAngle',
 'VerticalTEC',
 'AltIntJouleHeating(W/m2)',
 'AltIntHeatingTransfer(W/m2)',
 'AltIntEuvHeating(W/m2)',
 'AltIntPhotoElectronHeating(W/m2)',
 'AltIntChamicalHeating(W/m2)',
 'AltIntRadCooling(W/m2)',
 'AltIntCO2Cooling(W/m2)',
 'AltIntNOCooling(W/m2)',
 'AltIntOCooling(W/m2)']

In [67]:
for i, n in enumerate(gitmvars):
    if 'Heat' in n or 'Cool' in n:
        print(i,n)

3 AltIntJouleHeating(W/m2)
4 AltIntHeatingTransfer(W/m2)
5 AltIntEuvHeating(W/m2)
6 AltIntPhotoElectronHeating(W/m2)
7 AltIntChamicalHeating(W/m2)
8 AltIntRadCooling(W/m2)
9 AltIntCO2Cooling(W/m2)
10 AltIntNOCooling(W/m2)
11 AltIntOCooling(W/m2)


In [40]:
gitmvars[3], gitmvars[5]

('AltIntJouleHeating(W/m2)', 'AltIntEuvHeating(W/m2)')

In [84]:
nrows = 2
ncols = 2
t_index = 14

dtime = gitm_dtimes[t_index]

# Do it automatically:
nvars = [3,5,6,7]
nrows=len(nvars)
data = []
titles=['North','South']
colorbar_labels=[]

for var in nvars:
    data.append(gitmbins[t_index,var,:,:,:].reshape([len(lons), len(lats)]))
    colorbar_labels.append(gitmvars[var])
                


"""
data=[0,0] 
# Don't set data in Column! 
# We'll plot same data, other hemisphere in the columns
data[0] = gitmbins[t_index,3,:,:,:].reshape([len(lons), len(lats)])
data[1] = gitmbins[t_index,5,:,:,:].reshape([len(lons), len(lats)])


titles=[0,0]
titles[0] = "North"
titles[1] = "South"
# Only needed in first row!


colorbar_labels=[0 for i in range(nrows)]
"""


# fig = plt.figure(figsize=(10, 8.5))
fig, axes = plt.subplots(nrows,ncols, figsize=(10,4*nrows),subplot_kw={'projection':'polar'})



fig.suptitle('Heating/Cooling %s UT from storm onset \n (%s UT)\n\n'
                % (UT_from_Storm_onset(
                    dtime, dtime_storm_start),
                str(dtime)), fontsize=15)



for row in range(nrows):
    for col in range(ncols):
        
        r, theta = np.meshgrid(90-lats[maskNorth], lons)
        
        if col == 0:
            axes[row,col].pcolor(np.deg2rad(theta), r,
                             data[row][:, maskNorth].copy(),
                             vmin=np.min(data[row]),
                             vmax=np.max(data[row]),)
        elif col ==1:
            cb = axes[row,col].pcolor(np.deg2rad(theta), r,
                             data[row][:, maskNorth].copy(),
                             vmin=np.min(data[row]),
                             vmax=np.max(data[row]),)
            
            fig.colorbar(cb, label=colorbar_labels[row])
        
        
        ylabels = ['80', '70', '60', '50']
        axes[row,col].set_yticklabels(ylabels)
        axes[row,col].set_xticks(np.arange(0, 2*np.pi, np.pi/2))
        axes[row,col].set_yticks(np.arange(10, 50, 10))
        
        if row == 0:
            axes[row,col].set_title(titles[col])
        
        

fig.tight_layout()

# plt.show()
plt.savefig('some-heatingcooling-during-storm')
plt.close()

/glade/scratch/abukowski/ipykernel_61115/170109672.py:70: UserWarning: FixedFormatter should only be used together with FixedLocator
  axes[row,col].set_yticklabels(ylabels)


## GITM Rho Maps

In [128]:
gitm_dtimes, gitmgrid, gitmbins, gitmvars = GITM.read_gitm_into_nparrays(
    "/glade/u/home/abukowski/scratch/GITM-simstorm-run1/run2/UA/data", dtime_storm_start, t_start_idx=1, t_end_idx=3,return_vars=True)

100%|██████████| 48/48 [02:18<00:00,  2.88s/it]


In [129]:
gitm_dtimes[14], gitm_dtimes[20], gitm_dtimes[44], gitmvars[0]

(datetime.datetime(2011, 5, 21, 12, 10),
 datetime.datetime(2011, 5, 21, 12, 40),
 datetime.datetime(2011, 5, 21, 14, 40),
 'Rho')

In [130]:
alt = 225 # km
times = [14,20,26,32,38,44]
var = 0

nrows = 3
ncols = 2


alt_idx = np.argmin(np.abs(np.unique(gitmgrid['altitude']) - alt*1000))


In [131]:
fits = filters.make_fits(gitmbins)
percents = 100*(gitmbins - fits)/gitmbins

del fits, gitmbins


In [133]:
data = [[],[],[]]
titles = [[],[],[]]

for row in range(nrows):
    for col in range(ncols):
        data[row].append(percents[times[row+col],var,:,:,alt_idx].copy())
        titles[row].append(plot_help.UT_from_Storm_onset(gitm_dtimes[times[row+col]], dtime_storm_start))

del percents

In [137]:
titles[0][0] += ' UT from Storm Onset'

In [141]:

fig, axes = plt.subplots(nrows,ncols, figsize=(10,3*nrows), sharex=True, sharey=True)

for row in range(nrows):
    for col in range(ncols):
        im = pr.draw_map(data[row][col], [-2,2], save_or_show="return", ax=axes[row,col], title=str(row), cmap='bwr')
        
        axes[row,col].set_title(titles[row][col])
        
        if col == 0:
            axes[row,col].set_ylabel("Latitude (deg)")

        if row == nrows-1:
            axes[row,col].set_xlabel("Longitude (deg)")


fig.suptitle('Neutral Density Perturbations from GITM at 225 km          ', fontsize=16, ha='center')
# spaces needed to center properly with the colorbar existing.
    
fig.colorbar(im, ax=axes.ravel().tolist(), orientation="vertical",
             label = '% over Background', aspect=75)
fig.align_xlabels()


plt.savefig("/glade/u/home/abukowski/figures/fig1")
# plt.show()
plt.close()

In [146]:
alts

[225, 663]

In [142]:
gitm_dtimes, gitmgrid, gitmbins, gitmvars = GITM.read_gitm_into_nparrays(
    "/glade/u/home/abukowski/scratch/GITM-simstorm-run1/run2/UA/data", dtime_storm_start, t_start_idx=2, t_end_idx=8,return_vars=True, cols='Rho')

100%|██████████| 120/120 [05:29<00:00,  2.75s/it]


In [171]:
nrows=2
ncols=2

alts=[225,663]
lons=[1,-90]

In [172]:
alts_real = []
alt_idxs = []
lons_real = []
lon_idxs = []

for i in range(len(alts)):
    altidx=np.argmin(np.abs(np.unique(gitmgrid['altitude']) - alts[i]*1000))
    alt_idxs.append(altidx)
    alts_real.append(np.unique(gitmgrid['altitude'])[altidx])
    
    lonidx=np.argmin(np.abs(np.unique(gitmgrid['longitude']) - lons[i]))
    lon_idxs.append(lonidx)
    lons_real.append(np.unique(gitmgrid['longitude'])[lonidx])
    

In [163]:
fits = filters.make_fits(gitmbins)
percents = 100*(gitmbins - fits)/gitmbins

# del fits, gitmbins

In [173]:
data = [[],[]]
titles = [[],[]]

for row in range(nrows):
    for col in range(ncols):
        data[row].append(percents[:,var,lon_idxs[row],:,alt_idxs[col]].copy())
        titles[row].append('lon= %i deg, alt= %i km' %(lons_real[row], alts_real[col]/1000))

# del percents

In [175]:
fig, axes = plt.subplots(nrows,ncols, figsize=(10,3*nrows), sharex=True, sharey=True)

for row in range(nrows):
    for col in range(ncols):
        im = pr.draw_map(data[row][col], [-2,2], save_or_show="return", ax=axes[row,col], title=str(row), cmap='bwr')
        
        axes[row,col].set_title(titles[row][col])
        
        if col == 0:
            axes[row,col].set_ylabel("Latitude (deg)")

        if row == nrows-1:
            axes[row,col].set_xlabel("Longitude (deg)")


fig.suptitle('Keograms of Neutral Density Perturbations from GITM          ', fontsize=16, ha='center')
# spaces needed to center properly with the colorbar existing.
    
fig.colorbar(im, ax=axes.ravel().tolist(), orientation="vertical",
             label = '% over Background', aspect=75)
fig.align_xlabels()


plt.savefig("/glade/u/home/abukowski/figures/fig2")
# plt.show()
plt.close()

In [233]:
samidata, times = SAMI.read_sami_dene_tec(
    "/glade/u/home/abukowski/scratch/GITM-simstorm-run1/sami-gitm-coupled/",
    datetime.datetime(2011,5,20), dtime_storm_start,)

In [234]:
start = np.argmin(np.abs(times-datetime.datetime(2011,5,21,10))/pd.Timedelta(1, 'minute'))
end = np.argmin(np.abs(times-datetime.datetime(2011,5,21,20))/pd.Timedelta(1, 'minute'))

In [289]:
end, start

(528, 408)

In [312]:
lons=[90,-178,2,-90]

In [313]:
fitssami = filters.make_fits(samidata['data']['tec'])
percent=100*(samidata['data']['tec']-fitssami)/samidata['data']['tec']

grid=samidata['grid']

del samidata, fitssami

NameError: name 'samidata' is not defined

In [314]:
percent.shape

(625, 80, 100)

In [315]:
glons = []
glats = []
nlons = grid['glon'].shape[0]
nlats = grid['glat'].shape[2]
for lat in range(nlats):
    for lon in range(nlons):
        glon_i = grid['glon'][lon,0,lat]
        if glon_i > 180:
            glons.append(glon_i - 360)
        else:
            glons.append(glon_i)
        glats.append(grid['glat'][lon,0,lat])

glons = np.array(glons)


In [316]:
glons.shape

(8000,)

In [320]:
data = [[],[]]
titles = [[],[]]

i=0

for row in range(nrows):
    for col in range(ncols):
        sel_pts = np.where(
                    (np.abs(glons - lons[i]) < 3) &
                    (np.abs(glats) < 65) )[0]
        raw = []
        for t in range(start, end):
            raw.append(percent[t,:,:].copy().T.flatten()[sel_pts])
        data[row].append(np.array(raw))
        titles[row].append('lon= %i deg' %(lons[i]))
        
        i+=1
        
      

In [321]:
len(data), len(data[0]), data[0][0].shape

(2, 2, (120, 115))

In [326]:
fig, axes = plt.subplots(nrows,ncols, figsize=(10,3*nrows), sharex=True, sharey=True)

for row in range(nrows):
    for col in range(ncols):
        im = pr.make_a_keo(data[row][col], cbarlims=[-.25,.25],
                           save_or_show="return", ax=axes[row,col], cmap='bwr', extent=[-2,8,-90,90],
                          ylims=[-65,65])
        
        axes[row,col].set_title(titles[row][col])
        
        if col == 0:
            axes[row,col].set_ylabel("Latitude (deg)")

        if row == nrows-1:
            axes[row,col].set_xlabel("Hours from Storm Onset")


fig.suptitle('Keograms of TEC Perturbations from SAMI3          ', fontsize=16, ha='center')
# spaces needed to center properly with the colorbar existing.
    
fig.colorbar(im, ax=axes.ravel().tolist(), orientation="vertical",
             label = 'deltaTEC (TEC units)', aspect=75)
fig.align_xlabels()


plt.savefig("/glade/u/home/abukowski/figures/fig4")
# plt.show()
plt.close()

In [332]:
samidata, times = SAMI.read_sami_data(
    "/glade/u/home/abukowski/scratch/GITM-simstorm-run1/sami-gitm-coupled/",
    datetime.datetime(2011,5,20), dtime_storm_start, 1, 3,  cols=['edens'])

(80, 72, 256, 48) 48 420 468


In [344]:
samidata['data']['edens'].shape

(80, 72, 256, 48)

In [333]:
mlon=317
plottimes = [14,20,26,32,38,44]

In [347]:
mlons = np.unique(samidata['grid']['mlon']).round(2)
mlon_idx = np.argmin(np.abs(mlons-mlon))

mask = (samidata['grid']['mlon'].round(2) == mlons[mlon_idx])

x = samidata['grid']['mlat'][mask]
y = samidata['grid']['alt'][mask]

In [376]:
filt.shape

(18432, 48)

In [366]:
def make_filter(params=None):
    # Define the cutoff frequencies
    lowcut = 1/(100/60)  # 100 minutes in units of sample^-1
    highcut = 1/(30/60)  # 30 minutes in units of sample^-1

    # Define the Butterworth filter
    nyquist = 0.5 * 5  # 5 minutes is the sampling frequency
    low = lowcut / nyquist
    high = highcut / nyquist
    sos = signal.butter(2, [low, high], btype='bandstop', output='sos')
    return sos


def remove_background(time_series, sos, axis = 0):

    # Apply the filter to the time series
    filtered_data = signal.sosfiltfilt(sos, time_series, axis)

    return filtered_data

In [370]:
sos = make_filter()
filt = remove_background(samidata['data']['edens'][mask, :].copy(), sos, axis=1)

percent = 100*(samidata['data']['edens'][mask,:] - filt)/samidata['data']['edens'][mask,:]


In [378]:
data = [[],[], []]
titles = [[],[],[]]

i=0

for row in range(nrows):
    for col in range(ncols):

        raw = []
        data[row].append(percent[:,plottimes[i]])
        
        dtime=times[plottimes[i]]
        
        mlt = aacgmv2.wrapper.convert_mlt(
                        mlon, dtime, m2a=False).round(2)[0]
        ut=UT_from_Storm_onset(dtime, dtime_storm_start)
        
        titles[row].append('MLT = %s, %s from Storm Onset' %(str(mlt), ut))
        
        i+=1

In [379]:
data[0][0].shape

(18432,)

In [380]:
nrows=3
ncols=2

In [392]:
fig, axes = plt.subplots(nrows,ncols, figsize=(10,3*nrows), sharex=True, sharey=True)

for row in range(nrows):
    for col in range(ncols):
        im = pr.draw_field_line_plot(x, y, data[row][col], 
                                     interpolate=True, ax = axes[row,col],
                                     save_or_show='return', cbar_lims=[-3,3], 
                                    cmap='bwr')
        
        axes[row,col].set_title(titles[row][col])
        
        if col == 0:
            axes[row,col].set_ylabel("Altitude (km)")

        if row == nrows-1:
            axes[row,col].set_xlabel("Mlat (deg)")


fig.suptitle('Keograms of TEC Perturbations from SAMI3     \n Along 317.25 degrees Magnetic Longitude       ', fontsize=16, ha='center')
# spaces needed to center properly with the colorbar existing.
    
fig.colorbar(im, ax=axes.ravel().tolist(), orientation="vertical",
             label = 'deltaTEC (TEC units)', aspect=75)
fig.align_xlabels()


plt.savefig("/glade/u/home/abukowski/figures/fig5")
# plt.show()
plt.close()